<a href="https://colab.research.google.com/github/arresejo/misc/blob/main/deepdream.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import random

from keras.models import Sequential, load_model, clone_model
from keras.layers import Dense, Activation
from keras.losses import Huber
from tensorflow.keras.optimizers import Adam



In [2]:
import gym

!pip3 install box2d-py
!pip3 install gym[Box_2D]

     |████████████████████████████████| 448 kB 4.5 MB/s 


In [3]:
env_name = "LunarLander-v2"
env = gym.make(env_name)

In [ ]:
#https://github.com/rogerxcn/lunar_lander_project/blob/master/dqn_agent_keras.py

import numpy as np
import keras
from keras.activations import relu, linear
#import lunar_lander as lander
from collections import deque
import gym
import random
#from keras.utils import to_categorical
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam



learning_rate = 0.001
model = keras.Sequential()
model.add(keras.layers.Dense(64, input_dim=8, activation=relu))
model.add(keras.layers.Dense(64, activation=relu))
model.add(keras.layers.Dense(4, activation=linear))
# model.compile(loss="mse", optimizer=keras.optimizers.adam(lr=learning_rate ))
model.compile(loss="mse", optimizer=Adam(lr=learning_rate))

epsilon = 1
gamma = .99
batch_size = 64
memory = deque(maxlen=1000000)
min_eps = 0.01
model = model

def replay_experiences():
    if len(memory) >= batch_size:
        sample_choices = np.array(memory)
        mini_batch_index = np.random.choice(len(sample_choices), batch_size)
        #batch = random.sample(memory, batch_size)
        states = []
        actions = []
        next_states = []
        rewards = []
        finishes = []
        for index in mini_batch_index:
            states.append(memory[index][0])
            actions.append(memory[index][1])
            next_states.append(memory[index][2])
            rewards.append(memory[index][3])
            finishes.append(memory[index][4])
        states = np.array(states)
        actions = np.array(actions)
        next_states = np.array(next_states)
        rewards = np.array(rewards)
        finishes = np.array(finishes)
        states = np.squeeze(states)
        next_states = np.squeeze(next_states)
        q_vals_next_state = model.predict_on_batch(next_states)
        q_vals_target = model.predict_on_batch(states)
        max_q_values_next_state = np.amax(q_vals_next_state, axis=1)
        q_vals_target[np.arange(batch_size), actions] = rewards + gamma * (max_q_values_next_state) * (1 - finishes)
        model.fit(states, q_vals_target, verbose=0)
        global epsilon
        if epsilon > min_eps:
            epsilon *= 0.996

if __name__ == '__main__':
    #env = lander.LunarLander()
    # env.seed(0)
    num_episodes = 400
    np.random.seed(0)
    scores  = []
    for i in range(num_episodes+1):
        score = 0
        state = env.reset()
        finished = False
        if i != 0 and i % 50 == 0:
            model.save(".\saved_models\model_"+str(i)+"_episodes.h5")
        for j in range(3000):
            state = np.reshape(state, (1, 8))
            if np.random.random() <= epsilon:
                action =  np.random.choice(4)
            else:
                action_values = model.predict(state)
                action = np.argmax(action_values[0])

            #env.render()
            next_state, reward, finished, metadata = env.step(action)
            next_state = np.reshape(next_state, (1, 8))
            memory.append((state, action, next_state, reward, finished))
            replay_experiences()
            score += reward
            state = next_state
            if finished:
                scores.append(score)
                print("Episode = {}, Score = {}, Avg_Score = {}".format(i, score, np.mean(scores[-100:])))
                break


/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:33: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.


Episode = 0, Score = -106.33521900614146, Avg_Score = -106.33521900614146
Episode = 1, Score = -104.79358080494598, Avg_Score = -105.56439990554372
Episode = 2, Score = -41.14642312892306, Avg_Score = -84.09174098000351
Episode = 3, Score = -167.6868918438088, Avg_Score = -104.99052869595482
Episode = 4, Score = -239.60025151031925, Avg_Score = -131.9124732588277
Episode = 5, Score = -98.26193079864593, Avg_Score = -126.30404951546409
Episode = 6, Score = -164.03931991723636, Avg_Score = -131.69480243000297
Episode = 7, Score = 7.07589507365708, Avg_Score = -114.34846524204548
Episode = 8, Score = -41.4842301030514, Avg_Score = -106.25243911549059
Episode = 9, Score = 36.79992763292415, Avg_Score = -91.94720244064911
Episode = 10, Score = -28.21459183408425, Avg_Score = -86.15332874914321
Episode = 11, Score = -99.08590750417791, Avg_Score = -87.2310436453961
Episode = 12, Score = -88.4108733904923, Avg_Score = -87.32179977963428
Episode = 13, Score = -189.79963847066387, Avg_Score = -

In [5]:
class Agent():
  def __init__(self, env, batch_size, gamma, alpha, epsilon, epsilon_decay, epsilon_min, update_frequency, model_fname, memory_size=1000000):
    self.env = env
    if not self.env.continuous:  # discrete
      self.num_actions = 4  # line 170 in source code: https://github.com/openai/gym/blob/master/gym/envs/box2d/lunar_lander.py
    else:
      return f"you still did not work on coninuous..."
    self.memory_size = 50000
    memory_space = 8+1+1+8+1  # states, action, reward, next_state, terminal   # TDL (soft-code it from the env)
    self.memory_counter = 0
    self.memory = np.zeros((memory_space, self.memory_size))
    self.batch_size = batch_size
    self.gamma = gamma
    self.alpha = alpha
    # self.num_episodes = num_episodes  # in the train method
    self.epsilon = epsilon
    self.epsilon_decay = epsilon_decay
    self.epsilon_min = epsilon_min
    self.update_frequency = update_frequency
    self.model_fname = model_fname

  def agent_model(self, layer_list, input_dims, lr, loss, optimizer):  # fcn
    model = Sequential()
    # for l_name, l_size, l_activation in layer_list[1:]:
    #   model.add(l_name(l_size, activation=l_activation))
    for l in layer_list:
      model.add(l)
    model.compile(optimizer=optimizer(learning_rate=lr), loss=loss)

    print(model.summary())

    return model
  
  # def model_fit(self):
  #   pass

  def store_in_memory(self, sarst_list):  # sarst_list := (state,action,next_state,reward,terminal) 
    # store transition (S, A, R, S', terminal) in replay memory D
    self.memory[:,self.memory_counter%self.memory_size] = np.concatenate(tuple(np.array(x,dtype=float,ndmin=1) for x in sarst_list))
    self.memory_counter += 1
  
  def choose_action(self, state):
    if np.random.random() > self.epsilon:
      action = np.argmax(self.model_q1.predict(state.reshape((1,8))))  # epsilon-greedy
    else:
      action = np.random.randint(self.num_actions)
    return action

  def get_rand_sample(self, memo):
    # return memo[:,np.random.choice(memo.shape[1], size=self.batch_size, replace=False)]
    # return memo[:,np.random.choice(memo.shape[0], size=self.batch_size, replace=False)]
    # print(memo.shape)
    # print(memo.shape[0], self.batch_size)

    out = memo[:,np.random.choice(memo.shape[1], size=self.batch_size, replace=False)]
    # print(out.shape)
    return out
  
  def train(self, num_episodes, layer_list, input_dims=8, loss='mse', optimizer=Adam):  # TDL: default of input_dims as num of states
    self.model_q1 = self.agent_model(layer_list, input_dims, self.alpha, loss, optimizer)

    self.model_q2 = clone_model(self.model_q1)
    self.model_q2.set_weights(self.model_q1.get_weights())
    reward_steps_hist = []  # tupple per episode:  (reward, num_steps)

    for episode in range(num_episodes+1):
      total_reward = 0
      steps = 0
      #terminal = False
      state = self.env.reset()  # seed=seed
      #while not terminal:
      for _ in range(3000):
        action = self.choose_action(state)
        next_state, reward, terminal, _ = self.env.step(action)
        ls=[state, action, reward,next_state,terminal]
        self.store_in_memory(ls)
        total_reward += reward
        if self.memory_counter > self.batch_size:

          batch = self.get_rand_sample(self.memory[:,:np.min((self.memory_counter, self.memory_size))])
          # y_state = batch[0:8, :]
          prediction_q = self.model_q1.predict(batch[0:8, :].T)  # predict the state from batch

          batch_index = np.arange(self.batch_size, dtype=np.int32)
          prediction_q[batch_index, np.array(batch[8,:], dtype=np.int32)] = \
              batch[9,:] + (batch[-1,:]==0)*self.gamma*np.max(self.model_q2.predict(batch[10:18,:].T),axis=1)  # batch[-1,:]==0 if terminal  (if terminal else from the pseudocode)
          self.model_q1.fit(batch[0:8,:].T, prediction_q, verbose=0)
          if steps%self.update_frequency==0:
            self.model_q2.set_weights(self.model_q1.get_weights())
        state = next_state
        # self.epsilon = np.max(((self.epsilon * self.epsilon_decay), self.epsilon_min))

        # optional: decay learning rate
        
        steps += 1
      if episode % 10 == 0:
        self.save_model()
      self.epsilon = np.max(((self.epsilon * self.epsilon_decay), self.epsilon_min))
      print((episode, total_reward, steps, self.epsilon))  

      reward_steps_hist.append((total_reward, steps))

    
  def save_model(self):
    self.model_q2.save(self.model_fname+".h5")
    




In [6]:
batch_size = 64

gamma = .99
alpha = .001

epsilon = 1
epsilon_decay = .996
epsilon_min = .01

update_frequency = 10  # in steps
memory_size = 1000000
num_episodes = 400

layers_list = [Dense(64, input_shape=(8,), activation='relu'), Dense(64, activation='relu'), Dense(4, activation='linear')]  # TDL: soft-code inut_shape and num_actions (as part of the class)

model_fname = 'DQN'
agent1 = Agent(env, batch_size, gamma, alpha, epsilon, epsilon_decay, epsilon_min, update_frequency, model_fname, memory_size=1000000)
agent1.train(num_episodes, layers_list, input_dims=8, loss=Huber(delta=1.0), optimizer=Adam)

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 64)                576       
                                                                 
 dense_4 (Dense)             (None, 64)                4160      
                                                                 
 dense_5 (Dense)             (None, 4)                 260       
                                                                 
Total params: 4,996
Trainable params: 4,996
Non-trainable params: 0
_________________________________________________________________
None
(0, -290888.99820025836, 3000, 0.996)
(1, -292819.23856116843, 3000, 0.992016)


KeyboardInterrupt: ignored